# 你有 Gurobi WLS 许可证吗?
本笔记本可以使用有限许可证运行,但详细程度会较低。如果你有 Gurobi WLS 许可证,可以[点击这里](https://colab.research.google.com/github/Gurobi/modeling-examples/blob/master/price_optimization/price_optimization_gurobiML_wls.ipynb)查看适用于该许可证的版本。

如果你没有,也别担心!你仍然可以使用免费的有限 Gurobi 许可证来完成这个版本。

# 第二部分:使用数学优化进行牛油果定价和供应

这是价格优化示例的第二部分:[多少钱才算太贵？使用数学优化进行牛油果定价和供应](https://github.com/Gurobi/modeling-examples/tree/master/price_optimization)

在第一部分中,使用了普通线性回归模型(OLS)来建立基于数据的价格和需求之间的关系。第二部分使用训练好的 `Scikit-learn` 模型替代 OLS 模型,并使用 [Gurobi Machine Learning](https://gurobi-machinelearning.readthedocs.io/en/stable/#) 包将其嵌入到 Gurobi 优化模型中。

在本示例中,我们还将使用 `gurobipy-pandas`,这是另一个 Gurobi 开源包,作为 pandas 与 gurobipy 连接的便捷(且可选)包装器。

如果你已经熟悉另一个笔记本中的示例,可以直接跳转到[构建回归模型](#Part-2:-Predict-the-Sales)部分,然后跳转到[构建优化问题](#Part-3:-Optimize-for-Price-and-Supply-of-Avocados)。

**目标**:开发一个用于牛油果定价和分销以实现收益最大化的数据科学和决策管道。

为实现这个目标,本笔记本将分三个阶段进行:

1. 快速回顾 [Hass Avocado Board](https://hassavocadoboard.com/) (HAB) 数据
2. 构建一个牛油果需求预测模型,该模型是价格、地区、年份和季节性的函数。
3. 设计一个优化模型,在考虑运输和成本的同时,设定最佳价格和供应数量以最大化净收益。

## 加载包并准备数据集

和第一个示例一样,我们使用真实的 HAB 销售数据。


In [1]:
import pandas as pd
import warnings
import numpy as np

HAB 数据集包含 2019-2022 年的销售数据。这些数据通过之前从 HAB 下载的 2015-2018 年销售数据进行了补充,这些数据可在 [Kaggle](https://www.kaggle.com/datasets/timmate/avocado-prices-2020) 上获取。

本笔记本将跳过第一个版本示例中的大部分数据预处理步骤。

In [2]:
data_url = "https://raw.githubusercontent.com/Gurobi/modeling-examples/master/price_optimization/"
avocado = pd.read_csv(data_url+"HAB_data_2015to2022.csv")
avocado["date"] = pd.to_datetime(avocado["date"])
avocado = avocado.sort_values(by="date")
avocado

,date,units_sold,price,region,year,month,peak
0,2015-01-04,3.382800,1.020000,Great_Lakes,2015,1,0
1,2015-01-04,2.578275,1.100000,Midsouth,2015,1,0
2,2015-01-04,5.794411,0.890000,West,2015,1,0
3,2015-01-04,3.204112,0.980000,Southeast,2015,1,0
4,2015-01-04,0.321824,1.050000,Northern_New_England,2015,1,0
...,...,...,...,...,...,...,...
3396,2022-05-15,0.445830,1.513707,Northern_New_England,2022,5,1
3397,2022-05-15,4.150433,1.269883,SouthCentral,2022,5,1
3398,2022-05-15,4.668815,1.644873,Northeast,2022,5,1
3399,2022-05-15,32.745321,1.527357,Total_US,2022,5,1


上述数据框中的一个地区是 `Total_US`，因此我们可以创建一个不包含总计的地区列表，该列表可以用于现在对数据进行子集划分。该列表稍后在示例中也会用到。

In [3]:
regions = [
    "Great_Lakes",
    "Midsouth",
    "Northeast",
    "Northern_New_England",
    "SouthCentral",
    "Southeast",
    "West",
    "Plains"
]
df = avocado[avocado.region.isin(regions)]

## 预测销量

在本示例的第一个实例中，对输入数据进行了进一步分析并做了一些可视化。在这里，我们将直接进行预测模型训练，首先将数据集随机分为 80% 的训练集和 20% 的测试集。

In [4]:
from sklearn.model_selection import train_test_split

X = df[["region", "price", "year", "peak"]]
y = df["units_sold"]
# 将数据分为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, random_state=1
)

注意，地区是一个分类变量，我们将使用 Scikit Learn 的 `OneHotEncoder` 来转换该变量。我们还使用标准缩放器对价格和年份索引进行处理，并使用 `make_column_transformer` 将所有这些组合到 `Column Transformer` 中。

In [5]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

feat_transform = make_column_transformer(
    (OneHotEncoder(drop="first"), ["region"]),
    (StandardScaler(), ["price", "year"]),
    ("passthrough", ["peak"]),
    verbose_feature_names_out=False,
    remainder='drop'
)

回归模型是由 `Column Transformer` 和我们想要用于回归的模型类型组成的管道。为了进行比较，我们将继续使用线性回归。

In [6]:
reg = make_pipeline(feat_transform, LinearRegression())
reg.fit(X_train, y_train)

# Get R^2 from test data
y_pred = reg.predict(X_test)
print(f"The R^2 value in the test set is {np.round(r2_score(y_test, y_pred),5)}")

The R^2 value in the test set is 0.9083


我们可以观察到测试集中有一个不错的 $R^2$ 值。现在我们将对完整数据集进行拟合训练。

In [7]:
reg.fit(X, y)

y_pred_full = reg.predict(X)
print(f"The R^2 value in the full dataset is {np.round(r2_score(y, y_pred_full),5)}")

The R^2 value in the full dataset is 0.90667


## 优化牛油果价格和供应

以下是数学优化模型表述的符号快速回顾。下标 $r$ 将用于表示每个地区。
### 输入参数
- $d(p,r)$：当牛油果价格为 $p$ 时地区 $r$ 的预测需求
- $B$：可以分配给各地区的牛油果总量
- $c_{waste}$：每个浪费的牛油果的成本 ($\$$)
- $c^r_{transport}$：将牛油果运输到地区 $r$ 的成本 ($\$$)
- $a_{min},a_{max}$：每个牛油果的最低和最高价格 ($\$$)
  $r$
- $b^r_{min},b^r_{max}$：分配给地区 $r$ 的最少和最多牛油果数量

以下代码设置这些参数的值。您可以自由调整这些值，看看优化模型的解决方案会如何变化。


In [8]:
# 集合和参数
B = 35  # 牛油果供应总量

peak_or_not = 1  # 1表示旺季；0表示非旺季
year = 2022

c_waste = 0.1  # 浪费一个牛油果的成本($)

# 运输一个牛油果的成本
c_transport = pd.Series(
    {
        "Great_Lakes": 0.3,
        "Midsouth": 0.1,
        "Northeast": 0.4,
        "Northern_New_England": 0.5,
        "SouthCentral": 0.3,
        "Southeast": 0.2,
        "West": 0.2,
        "Plains": 0.2,
    }, name='transport_cost'
)
c_transport = c_transport.loc[regions]

a_min = 0  # 牛油果最低价格
a_max = 3  # 牛油果最高价格

# 从数据集获取价格和库存数量的上下界
data = pd.concat([c_transport,
                  df.groupby("region")["units_sold"].min().rename('min_delivery'),
                  df.groupby("region")["units_sold"].max().rename('max_delivery')], axis=1)

data

,transport_cost,min_delivery,max_delivery
Great_Lakes,0.3,2.063574,7.094765
Midsouth,0.1,1.845443,6.168572
Northeast,0.4,2.364424,8.836406
Northern_New_England,0.5,0.219690,0.917984
SouthCentral,0.3,3.687130,10.323175
Southeast,0.2,2.197764,7.810475
West,0.2,3.260102,11.274749
Plains,0.2,1.058938,3.575499


#### 安装并导入 Gurobi 包

In [10]:
# %pip install gurobipy_pandas
# %pip install gurobi-machinelearning
import gurobipy_pandas as gppd
from gurobi_ml import add_predictor_constr

### 为回归的固定特征创建数据框

我们现在开始创建优化模型中回归的输入，这些输入具有固定的特征。

我们使用 gurobipy-pandas，它有助于更轻松地使用 pandas 数据创建 gurobipy 模型。

首先，创建一个包含我们优化问题中固定特征的数据框。
它以地区为索引（我们想要使用一个回归来预测每个地区的需求），并具有 3 个对应于固定特征的列：

* `year`（年份）
* `peak`（使用 `peak_or_not` 的值）
* `region`（重复地区名称）。

让我们显示数据框以确保它是正确的。

In [11]:
feats = pd.DataFrame(
    data={
        "year": year,
        "peak": peak_or_not,
        "region": regions,
    },
    index=regions
)
feats

,year,peak,region
Great_Lakes,2022,1,Great_Lakes
Midsouth,2022,1,Midsouth
Northeast,2022,1,Northeast
Northern_New_England,2022,1,Northern_New_England
SouthCentral,2022,1,SouthCentral
Southeast,2022,1,Southeast
West,2022,1,West
Plains,2022,1,Plains


### 决策变量

现在让我们定义决策变量。在我们的模型中，我们想要存储每个地区的
牛油果价格和分配数量。我们还需要变量来跟踪预测会售出多少牛油果
以及预测会浪费多少。以下符号用于建模这些决策变量。

- $p$ 每个地区的牛油果价格 ($\$$)
- $x$ 供应给每个地区的牛油果数量
- $s$ 每个地区预测销售的牛油果数量
- $w$ 每个地区预测浪费的牛油果数量
- $d$ 每个地区的预测需求

所有这些变量都是使用 gurobipy-pandas 创建的，通过函数 `gppd.add_vars` 它们被赋予与 `data` 数据框相同的索引。

In [12]:
import gurobipy as gp

m = gp.Model("Avocado_Price_Allocation")

p = gppd.add_vars(m, data, name="price", lb=a_min, ub=a_max) # 每个地区的牛油果价格
x = gppd.add_vars(m, data, name="x", lb='min_delivery', ub='max_delivery') # 供应给每个地区的牛油果数量
s = gppd.add_vars(m, data, name="s") # 给定价格下每个地区的预测销售量
w = gppd.add_vars(m, data, name="w") # 每个地区的额外浪费量
d = gppd.add_vars(m, data, lb=-gp.GRB.INFINITY, name="demand") # 添加回归的变量

m.update()

# 显示其中一个变量
p

Set parameter LicenseID to value 2601452


Great_Lakes                      <gurobi.Var price[Great_Lakes]>
Midsouth                            <gurobi.Var price[Midsouth]>
Northeast                          <gurobi.Var price[Northeast]>
Northern_New_England    <gurobi.Var price[Northern_New_England]>
SouthCentral                    <gurobi.Var price[SouthCentral]>
Southeast                          <gurobi.Var price[Southeast]>
West                                    <gurobi.Var price[West]>
Plains                                <gurobi.Var price[Plains]>
Name: price, dtype: object

### 添加供应约束

现在我们介绍约束条件。第一个约束是确保供应的牛油果总数等于 $B$，这可以用数学表达式表示如下。

\begin{align*} \sum_{r} x_r &= B \end{align*}


In [13]:
m.addConstr(x.sum() == B)
m.update()

### 添加定义销售数量的约束

简单回顾一下，销售数量是分配数量和预测需求的最小值，即 $s_r = \min \{x_r,d_r(p_r)\}$。这个关系可以通过以下两个约束条件为每个地区 $r$ 建模。

\begin{align*} s_r &\leq x_r  \\
s_r &\leq d(p_r,r) \end{align*}

在这种情况下，我们使用 gurobipy-pandas 的 `add_constrs` 函数，基于上述不等式，这个函数的使用非常直观。

In [14]:
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, x)
gppd.add_constrs(m, s, gp.GRB.LESS_EQUAL, d)
m.update()

### 添加浪费约束

最后，我们应该定义每个地区的预测浪费量，它由预测不会售出的供应数量给出。
我们可以为每个地区 $r$ 用数学表达式表示这一点。

\begin{align*} w_r &= x_r - s_r \end{align*}

In [15]:
gppd.add_constrs(m, w, gp.GRB.EQUAL, x - s)
m.update()

### 添加预测需求的约束
首先，我们创建预测器约束的完整输入。我们将 `p` 变量和固定特征连接起来。请记住，预测的价格是地区、年份和旺季/淡季的函数。

In [16]:
m_feats = pd.concat([feats, p], axis=1)[["region", "price", "year", "peak"]]
m_feats

,region,price,year,peak
Great_Lakes,Great_Lakes,<gurobi.Var price[Great_Lakes]>,2022,1
Midsouth,Midsouth,<gurobi.Var price[Midsouth]>,2022,1
Northeast,Northeast,<gurobi.Var price[Northeast]>,2022,1
Northern_New_England,Northern_New_England,<gurobi.Var price[Northern_New_England]>,2022,1
SouthCentral,SouthCentral,<gurobi.Var price[SouthCentral]>,2022,1
Southeast,Southeast,<gurobi.Var price[Southeast]>,2022,1
West,West,<gurobi.Var price[West]>,2022,1
Plains,Plains,<gurobi.Var price[Plains]>,2022,1


现在，我们只需调用
[add_predictor_constr](https://gurobi-machinelearning.readthedocs.io/en/stable/api/AbstractPredictorConstr.html#gurobi_ml.add_predictor_constr)
来将连接特征和需求的约束插入到模型 `m` 中。

重要的是要保持上述列的顺序，否则您会看到错误。这些列必须与训练数据的顺序相同。

In [17]:
pred_constr = add_predictor_constr(m, reg, m_feats, d)
pred_constr.print_stats()

Model for pipe:
88 variables
24 constraints
Input has shape (8, 4)
Output has shape (8, 1)

Pipeline has 2 steps:

--------------------------------------------------------------------------------
Step            Output Shape    Variables              Constraints              
                                                Linear    Quadratic      General
col_trans            (8, 10)           24           16            0            0

lin_reg               (8, 1)           64            8            0            0

--------------------------------------------------------------------------------


In [18]:
m

<gurobi.Model Continuous instance Avocado_Price_Allocation: 49 constrs, 128 vars, Parameter changes: Username=(user-defined), LicenseID=2601452>

### 设置目标函数

目标是最大化**净收入**，即价格与数量的乘积减去所有地区的成本。该模型假设采购成本是固定的（因为数量 $B$ 是固定的），因此不纳入考虑。

使用已定义的决策变量，目标函数可以写成如下形式。

\begin{align} \textrm{最大化} &  \sum_{r}  (p_r * s_r - c_{waste} * w_r -
c^r_{transport} * x_r)& \end{align}

In [19]:
m.setObjective((p * s).sum() - c_waste * w.sum() - (c_transport * x).sum(),
               gp.GRB.MAXIMIZE)

### 启动求解器

在我们的模型中，由于我们取价格和预测销量的乘积，而这两者都是变量，因此目标函数是**二次的**。最大化二次项被称为**非凸**，
我们通过将 [Gurobi NonConvex 参数](https://www.gurobi.com/documentation/10.0/refman/nonconvex.html) 设置为 $2$ 来指定这一点。

In [20]:
m.Params.NonConvex = 2
m.optimize()

Set parameter NonConvex to value 2
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Non-default parameters:
NonConvex  2

Optimize a model with 49 rows, 128 columns and 184 nonzeros
Model fingerprint: 0xab60f9f8
Model has 8 quadratic objective terms
Coefficient statistics:
  Matrix range     [2e-01, 3e+00]
  Objective range  [1e-01, 5e-01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [2e-01, 2e+03]
  RHS range        [1e+00, 2e+03]
Presolve removed 24 rows and 96 columns

Continuous model is non-convex -- solving as a MIP

Presolve removed 32 rows and 104 columns
Presolve time: 0.00s
Presolved: 34 rows, 34 columns, 81 nonzeros
Presolved model has 8 bilinear constraint(s)
Variable types: 34 continuous, 0 integer (0 binary)
Found heuristic solution: objective 41.1671165

Root relaxation: object

求解器在不到一秒的时间内解决了优化问题。让我们现在通过将最优解存储在 Pandas 数据框中来分析它。

In [21]:
solution = pd.DataFrame(index=regions)

solution["Price"] = p.gppd.X
solution["Allocated"] = x.gppd.X
solution["Sold"] = s.gppd.X
solution["Wasted"] = w.gppd.X
solution["Pred_demand"] = d.gppd.X

opt_revenue = m.ObjVal
print("\n The optimal net revenue: $%f million" % opt_revenue)
solution.round(4)


 The optimal net revenue: $41.167116 million


,Price,Allocated,Sold,Wasted,Pred_demand
Great_Lakes,1.6139,3.5566,3.5566,0.0000,3.5566
Midsouth,1.5088,6.1686,3.5454,2.6231,3.5454
Northeast,1.9890,4.1629,4.1629,0.0000,4.1629
Northern_New_England,1.4412,0.9180,0.9180,0.0000,0.9180
SouthCentral,2.0027,4.4135,4.4135,0.0000,4.4135
Southeast,1.6964,5.5328,3.9588,1.5740,3.9588
West,2.1542,7.0820,4.9677,2.1143,4.9677
Plains,1.1521,3.1656,2.7593,0.4063,2.7593


我们还可以检查 Gurobi 解对回归模型的估计误差。

In [22]:
print(
    "Maximum error in approximating the regression {:.6}".format(
        np.max(pred_constr.get_error())
    )
)

Maximum error in approximating the regression 8.88178e-16


这是对使用 Gurobi Machine Learning 包的入门介绍。要了解更多关于这个示例的信息，请参见 [Github 上的价格优化示例](https://github.com/Gurobi/modeling-examples/tree/master/price_optimization)，
以及如何以交互方式使用该模型。

Copyright © 2023 Gurobi Optimization, LLC